In [45]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

In [74]:
# Create random input and output data
def init_toy_data(num_inputs, num_classes):
    np.random.seed(0)
    x = 10 * torch.randn(num_inputs, 1, 24, 256)
    y = np.random.randint(num_classes, size=num_inputs)
    print(y)
    print(np.histogram(y, bins=2))
    #y = np.array([0, 1, 2, 2, 1])
    return x, torch.from_numpy(y)

x, y = init_toy_data(2000,2)
#y = torch.zeros(2000, 2)#.random_(2)
#y[np.array(range(2000)),np.random.randint(0,2,2000)] = 1
print(x.shape)
print(y.shape)
print(x.dtype)

[0 1 1 ... 1 1 1]
(array([ 981, 1019]), array([0. , 0.5, 1. ]))
torch.Size([2000, 1, 24, 256])
torch.Size([2000])
torch.float32


In [6]:
print(torch.cuda.is_available())
#device = torch.device("cuda:0")

False


In [7]:
model = nn.Sequential(
                      nn.Conv2d(1,100,3),
                      nn.ReLU(),
                      nn.MaxPool2d(2, 2),
                      nn.Dropout(0.25),
                      nn.Conv2d(100,100,3),
                      nn.ReLU(),
                      nn.MaxPool2d(2, 2),
                      nn.Dropout(0.25),
                      nn.Conv2d(100,300,(2,3)),
                      nn.ReLU(),
                      nn.MaxPool2d(2, 2),
                      nn.Dropout(0.25),
                      nn.Conv2d(300,300,(1,7)),
                      nn.ReLU(),
                      nn.MaxPool2d((1,2), stride=1),
                      nn.Dropout(0.25),
                      nn.Conv2d(300,100,(1,3)),
                      nn.Conv2d(100,100,(1,3)),
                      nn.Flatten(),
                      nn.Linear(1900,6144),
                      nn.Linear(6144,2),
                      nn.Softmax())

pred = model(x)
loss_fn = torch.nn.CrossEntropyLoss()

/Users/dtyoung/miniconda3/envs/ml/lib/python3.8/site-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [51]:
print(pred.shape)
print(pred)

torch.Size([2000, 2])
tensor([[0.4876, 0.5124],
        [0.4892, 0.5108],
        [0.4917, 0.5083],
        ...,
        [0.4891, 0.5109],
        [0.4928, 0.5072],
        [0.4868, 0.5132]], grad_fn=<SoftmaxBackward>)


AxisError: axis 1 is out of bounds for array of dimension 1

In [10]:
#model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
n = x.shape[0]
epochs = 150
bs = 70
for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        start_i = i * bs
        end_i = start_i + bs
        xb = x[start_i:end_i]
        yb = y[start_i:end_i]
        # Forward pass: compute predicted y by passing x to the model.
        y_pred = model(xb)

        # Compute loss.
        loss = loss_fn(y_pred, yb)
        
        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update (which are the learnable
        # weights of the model). This is because by default, gradients are
        # accumulated in buffers( i.e, not overwritten) whenever .backward()
        # is called. Checkout docs of torch.autograd.backward for more details.
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        optimizer.step()
    print(epoch, loss.item())

AssertionError: Torch not compiled with CUDA enabled

In [52]:
y_pred = model(x)
print("error rate: ")
print(np.sum(np.abs(np.argmax(y_pred, axis=1)-y))/y.size)

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not numpy.ndarray